# 라이브러리 및 설정

In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from urllib.parse import urljoin
import requests
from selenium.webdriver.common.keys import Keys
import re
from PIL import Image
from io import BytesIO

# 이미지 저장 경로 설정
SAVE_DIR = os.path.join(os.getcwd(), 'images')
os.makedirs(SAVE_DIR, exist_ok=True)

# 1. 상품 리스트 수집

## 1) Selenium 브라우저 설정

In [2]:
# Selenium 브라우저 설정
browser = webdriver.Chrome()
url = 'https://www.amazon.com/-/ko/b?node=6936790011&ref=sr_ex_n_1'
# 아마존 페이지 열기 (언어 설정 한글)
browser.get( url +'&language=ko_KR')

In [3]:
# 페이지 소스 가져오기
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

## 2) 상품 리스트 수집

In [4]:
# 페이지 번호 가져오기
pagination_items = soup.select('span.s-pagination-item.s-pagination-disabled')
if pagination_items:
    pagination_item_text = pagination_items[-1].text
    if pagination_item_text == '이전':
        pagination_items = soup.select('a.s-pagination-item.s-pagination-button')
        if len(pagination_items) > 1:
            pagination_item_text = pagination_items[-2].text
        else:
            pagination_item_text = '1'  # 첫 페이지인 경우
else:
    pagination_items = soup.select('a.s-pagination-item.s-pagination-button')
    if pagination_items:
        pagination_item_text = pagination_items[-1].text
    else:
        pagination_item_text = '1'  # 첫 페이지인 경우

In [5]:
page_number = int(pagination_item_text)

results = []

In [6]:
# 최대 15페이지까지 데이터 수집 (495개까지 수집이 가능하나 광고 상품, 중복 상품 제외 시 300개 전후로 수집)
repeat_count = min(page_number, 15)

for _ in range(repeat_count):  # 첫 페이지도 포함하므로 그대로 사용
    # 상품 정보 정리하기
    product_part_list = soup.select('div.s-main-slot.s-result-list.s-search-results.sg-row > div.sg-col-4-of-24.sg-col-4-of-12.s-result-item.s-asin.sg-col-4-of-16.sg-col.s-widget-spacing-small.sg-col-4-of-20.gsx-ies-anchor')

    for product_part in product_part_list:
        sponsored_label = product_part.select('span.a-color-secondary.s-sponsored-label-text')
        if sponsored_label and sponsored_label[0].text.strip() == '후원':
            continue  # 광고 상품은 건너뜀        
        
        # 상품명 가져오기
        product_name = product_part.select('span.a-size-base-plus.a-color-base.a-text-normal')[0].text.strip().replace("\n", "")
        
        # 상품 URL 가져오기
        product_url = product_part.select('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')[0]
        relative_url = product_url['href']
        base_url = 'https://www.amazon.com'
        absolute_url = urljoin(base_url, relative_url)
        
        data = [product_name, absolute_url]
        results.append(data)
    
    # '다음' 버튼 클릭
    page_button = browser.find_elements('css selector', 'a.s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')
    if not page_button:
        break
    page_button[0].click()
    time.sleep(5)  # 페이지 로드 대기

    # 페이지 소스 업데이트
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')

# 브라우저 닫기
#browser.quit()

# 결과 출력
for result in results:
    print(result)


['Amazon Basics 비타민 D3 2000 IU 젤리, 오렌지, 레몬 및 딸기, 160개 (1회 제공량당 2개) (이전 Solimo)', 'https://www.amazon.com/-/ko/sspa/click?ie=UTF8&spc=MToyMDcxMzUwNjc5ODE4NDA3OjE3MzA1ODYxODM6c3BfYXRmOjIwMDAxMDUxMTI5MDEzMTo6MDo6&url=%2Fdp%2FB07BBNNHRX%2Fref%3Dsr_1_1_ffob_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.Q64FY_yJrN5RRZhI3hLeB7HIx-liMiTBNnNnYv7eJs7TqYwFecitM6YjhuKysECxLAvBWy3N1kcE2NDza4wYeK6WEfsVZwnZv6pPg44T7WXnmPrrRl1uUZf36QZS917mt1VgtxxBhpyuMINNuHIEbQCxubll2JZ5GzJxPwfQnm1KV0n64tok_ugB1LOdeJa7PFyvFhnOXtO_10fNpO9-NYvE0Gv-yCMlThOkJnY7rN5U45uJUS9ieEzg1MUa7eq9IF7udp8r0lEHJeuzFlAaxVm8TxS50iJ7N_GYuMLpybI.sWK5ncAmIAZpmN45CDxdXFRwHjbWXgH32FO0R9_Sbhk%26dib_tag%3Dse%26keywords%3D%25EB%25B9%2584%25ED%2583%2580%25EB%25AF%25BC%2B%25EB%25B3%25B4%25EC%25B6%25A9%25EC%25A0%259C%26qid%3D1730586183%26s%3Dhpc%26sr%3D1-1-spons%26ts_id%3D6936790011%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1']
['One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 E 및 아연 보충제, 면역 건강 지원, B12, 칼슘 등을 위한 보충제, 200개', 'https://www.amazon.com/-/ko/dp/B001

In [7]:
# 결과를 데이터프레임으로 저장
df = pd.DataFrame(results, columns=['상품명', '상품url'])
df = df.drop_duplicates(subset='상품명')
df = df.reset_index(drop=True)
df.to_csv('비타민_raw.csv', index=False, encoding='utf-8-sig')
print("상품 리스트 수집 완료")

상품 리스트 수집 완료


# 2. 상품 데이터 크롤링

## 1) CSV 파일 읽기

In [14]:
#수집된 상품 URL 로드
products_df = pd.read_csv('비타민_raw.csv')
products_df = products_df.head(50)
products_df

,상품명,상품url
0,"Amazon Basics 비타민 D3 2000 IU 젤리, 오렌지, 레몬 및 딸기,...",https://www.amazon.com/-/ko/sspa/click?ie=UTF8...
1,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",https://www.amazon.com/-/ko/dp/B001F71XAI/ref=...
2,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",https://www.amazon.com/-/ko/dp/B00YMSLT88/ref=...
3,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",https://www.amazon.com/-/ko/dp/B00GB85JR4/ref=...
4,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",https://www.amazon.com/-/ko/dp/B06WRNG37C/ref=...
5,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCD4D7F/ref=...
6,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,https://www.amazon.com/-/ko/dp/B07255MPRN/ref=...
7,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCCVL9V/ref=...
8,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",https://www.amazon.com/-/ko/dp/B0741FTPXD/ref=...
9,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",https://www.amazon.com/-/ko/dp/B00DS5BGDY/ref=...


In [15]:
# URL 리스트 생성
product_urls = products_df['상품url'].tolist()

# 브라우저 열기
browser = webdriver.Chrome()

results = []

## 2) ASIN 찾기 함수

In [16]:
def find_asin(soup):
    # 1. 테이블 형식에서 ASIN 찾기
    for row in soup.find_all('tr'):
        header = row.find('th', class_='prodDetSectionEntry')
        if header and 'ASIN' in header.text:
            return row.find('td', class_='prodDetAttrValue').text.strip()

    # 2. detailBullets_feature_div에서 ASIN 찾기
    detail_bullets = soup.find('div', id='detailBullets_feature_div')
    if detail_bullets:
        for list_item in detail_bullets.find_all('li'):
            header = list_item.find('span', class_='a-text-bold')
            if header and 'ASIN' in header.text:
                return list_item.find_all('span')[-1].text.strip()

    print("ASIN을 찾을 수 없습니다.")
    return None

## 3) 상품 정보 크롤링 함수

In [17]:
# 상품 정보 크롤링 함수 정의
def get_product_info(url, index, total):
    try:
        print(f"Processing {index + 1}/{total}")
        browser.get(url + '&language=ko_KR&currency=KRW')
        time.sleep(1)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        
                
        # 상품명
        title_elem = soup.select_one('span#productTitle')
        if not title_elem:
            print("찾을 수 없는 페이지입니다.")
            return

        # 상품 ID
        asin = find_asin(soup)
        if not asin:
            print("ASIN을 찾을 수 없습니다.")
            return
        
        title = title_elem.text.strip()

        #브랜드
        brand_elements = soup.select('tr.a-spacing-small.po-brand > td.a-span9 > span.a-size-base.po-break-word')
        if brand_elements:
            brand = brand_elements[0].text.strip()
        else:
            brand = ""
            
        # 정가
        original_price_elem = soup.select_one('span.basisPrice span.a-price.a-text-price > span.a-offscreen')
        if original_price_elem:
            original_price = float(original_price_elem.text.replace('US$', '').replace(',', ''))
        else:
            original_price = 0
            

        
        # 할인가
        price_whole_elem = soup.select_one('span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay > span> span.a-price-whole')
        price_fraction_elem = soup.select_one('span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay > span> span.a-price-fraction')
        
        if price_whole_elem and price_fraction_elem:
                    price_whole = price_whole_elem.text.strip().replace(',', '').replace('.', '')
                    price_fraction = price_fraction_elem.text.strip()
                    discounted_price = float(f"{price_whole}.{price_fraction}")
        else:
            discounted_price = 0


         # 평균별점 추출 및 실수로 변환
        avg_rating_elem = soup.select_one('a.a-popover-trigger.a-declarative > span.a-size-base.a-color-base')
        avg_rating = float(avg_rating_elem.text.strip()) if avg_rating_elem else ""

        
        # 평가 개수 추출
        rating_count_elem = soup.select_one('span#acrCustomerReviewText')
        if rating_count_elem:
            rating_count_text = rating_count_elem.text.strip()
            rating_count = int(re.sub(r'[^0-9]', '', rating_count_text))
        else:
            rating_count = ""
        

        page_button = browser.find_elements('css selector', 'li.a-spacing-small.item.imageThumbnail.a-declarative > span.a-list-item > span.a-button.a-button-thumbnail.a-button-toggle > span.a-button-inner > input.a-button-input')
        len(page_button)
        
        for i in range(len(page_button)):
            page_button[i].click()
        
        
        # 페이지 소스 가져오기
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        
        # 이미지 선택자로 img 태그들 가져오기
        imgs = soup.select('div.imgTagWrapper > img')
        
        # 각 이미지 태그의 src 속성값 리스트로 저장
        img_urls = [img['src'] for img in imgs]
        img_urls_paragraph = ' '.join(img_urls) if img_urls else ""

        
        # 품목 정보
        info_elements = soup.select('ul.a-unordered-list.a-vertical.a-spacing-mini > li.a-spacing-mini > span.a-list-item')
        info_text = ' '.join([elem.text.strip() for elem in info_elements])  if info_elements else ""

        
        # 상품 설명
        detail_texts = [elem.text.strip() for elem in soup.select('div#productDescription > p > span')]
        detail_paragraph = '/'.join(detail_texts) if detail_texts else ""

        
        # 카테고리
        category_texts = [elem.text.strip() for elem in soup.select('a.a-link-normal.a-color-tertiary')]
        category_paragraph = '/'.join(category_texts) if category_texts else ""
        
        results.append([asin, title, brand, original_price, discounted_price, avg_rating, rating_count, img_urls_paragraph, info_text, detail_paragraph, category_paragraph])
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

## 4) 전체 제품 정보 수집

In [18]:
# 전체 제품 정보 수집 함수 정의
def scrape_product_info(product_urls):
    total = len(product_urls)
    for index, url in enumerate(product_urls):
        get_product_info(url, index, total)

## 5) 상품 정보 크롤링 실행

In [19]:
scrape_product_info(product_urls)

Processing 1/50
Processing 2/50
Processing 3/50
Processing 4/50
Processing 5/50
Processing 6/50
Processing 7/50
Processing 8/50
Processing 9/50
Processing 10/50
Processing 11/50
Processing 12/50
Processing 13/50
Processing 14/50
Processing 15/50
Processing 16/50
Processing 17/50
Processing 18/50
Processing 19/50
Processing 20/50
Processing 21/50
Processing 22/50
Processing 23/50
Processing 24/50
Processing 25/50
Processing 26/50
Processing 27/50
Processing 28/50
Processing 29/50
Processing 30/50
Processing 31/50
Processing 32/50
Processing 33/50
Processing 34/50
Processing 35/50
Processing 36/50
Processing 37/50
Processing 38/50
Processing 39/50
Processing 40/50
Processing 41/50
Processing 42/50
Processing 43/50
Processing 44/50
Processing 45/50
Processing 46/50
Processing 47/50
Processing 48/50
Processing 49/50
Processing 50/50


In [20]:
# 데이터프레임 변환
detail_df = pd.DataFrame(results, columns=['asin', 'title', ' brand', 'original_price',
                                           'discounted_price', 'avg_rating', 'rating_count', 'img_urls_paragraph',
                                           'info_text',  'detail_paragraph', 'category_paragraph'])

# 결과 출력
detail_df

,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,img_urls_paragraph,info_text,detail_paragraph,category_paragraph
0,B07BBNNHRX,"Amazon Basics 비타민 D3 2000 IU 젤리, 오렌지, 레몬 및 딸기,...",Amazon Basics,0.00,0.00,4.7,19108,https://m.media-amazon.com/images/I/51sX+0vlXb...,"젤리 160개(80회분), 2개월 이상 분량(나열된 서빙 사이즈로 매일 섭취) 제품...","Amazon Basics 비타민 D 젤리는 오렌지, 레몬, 딸기 맛과 다른 천연 향...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
1,B001F71XAI,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",ONE A DAY,0.00,17.48,4.7,48490,https://m.media-amazon.com/images/I/616SPCJQAB...,One A Day 남성용 건강 포뮬라 멀티비타민은 심장의 건강과 같은 남성이 주로 ...,One A Day 남성용 건강 포뮬라 멀티비타민은 1개의 알로 다양한 주요 영양소를...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
2,B00YMSLT88,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",Nature Made,13.99,7.49,4.7,7463,https://m.media-amazon.com/images/I/71J+FRt+rp...,영양 지원 보충제: 철분이 함유된 Nature Made 멀티비타민 태블릿 130개입...,"Nature Made 철분 정제가 함유된 종합 비타민은 면역 건강, 근육 및 뼈 건...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
3,B00GB85JR4,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",NatureWise,0.00,14.99,4.7,174050,https://m.media-amazon.com/images/I/71JF8TOgXU...,가장 활성이 높은 형태: 인증된 유기농 올리브 오일에 제공되는 NatureWise ...,필수 비타민인 이유./비타민 D는 몸 전체의 세포에서 찾을 수 있습니다. 칼슘과 인...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
4,B06WRNG37C,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",Dr. Berg Nutritionals,37.99,35.50,4.7,14855,https://m.media-amazon.com/images/I/61TB8Sq3Hv...,"10,000IU의 비타민 D3 K2 보충제에는 비타민 D3 10,000IU(5,00...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
5,B09KCD4D7F,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",Centrum,19.99,15.99,4.7,40250,https://m.media-amazon.com/images/I/71kh-hE7hV...,"Centrum 실버 멀티비타민 남성용 50 플러스, 비타민 D3, B-비타민 및 아...",Centrum 남성용 실버 멀티비타민 50 플러스는 미량 영양소가 함유된 종합 비타...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
6,B07255MPRN,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,Sports Research,0.00,23.95,4.7,42932,https://m.media-amazon.com/images/I/512+wh1BkV...,"함께 더 좋음: ""햇빛 비타민""으로 알려진 비타민 D3는 칼슘 대사를 돕고 건강한 ...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
7,B09KCCVL9V,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",Centrum,19.29,15.66,4.8,19005,https://m.media-amazon.com/images/I/7150wfe2mj...,"Centrum Silver 여성용 멀티비타민 50 플러스 200개입 1병, 비타민 ...",Centrum Silver 여성용 멀티비타민 50 플러스는 전신 건강을 지원하는 미...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
8,B0741FTPXD,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",SmartyPants,32.99,24.76,4.5,14933,https://m.media-amazon.com/images/I/71Q4qdP6dg...,"포장은 다를 수 있습니다 - 신선하고 새로운 외관, 동일한 신뢰할 수 있는 품질 과...","SmartyPants Teen Guy Complete는 눈 건강을 위한 피쉬 오일,...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
9,B00DS5BGDY,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",Nature Made,23.89,9.98,4.7,33972,https://m.media-amazon.com/images/I/713dgE1NB6...,"Nature Made 비타민 B12 1000 mcg 소프트젤 150일 분량, 병당 ...",비타민 B12는 다른 비타민B와 마찬가지로 먹는 음식을 세포 에너지로 변환하는 데 ...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 B/B12"


## 5) 데이터 처리

In [21]:
# 가격이 없는 데이터 삭제
detail_df = detail_df[detail_df['discounted_price'] != 0]
detail_df

,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,img_urls_paragraph,info_text,detail_paragraph,category_paragraph
1,B001F71XAI,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",ONE A DAY,0.00,17.48,4.7,48490,https://m.media-amazon.com/images/I/616SPCJQAB...,One A Day 남성용 건강 포뮬라 멀티비타민은 심장의 건강과 같은 남성이 주로 ...,One A Day 남성용 건강 포뮬라 멀티비타민은 1개의 알로 다양한 주요 영양소를...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
2,B00YMSLT88,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",Nature Made,13.99,7.49,4.7,7463,https://m.media-amazon.com/images/I/71J+FRt+rp...,영양 지원 보충제: 철분이 함유된 Nature Made 멀티비타민 태블릿 130개입...,"Nature Made 철분 정제가 함유된 종합 비타민은 면역 건강, 근육 및 뼈 건...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
3,B00GB85JR4,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",NatureWise,0.00,14.99,4.7,174050,https://m.media-amazon.com/images/I/71JF8TOgXU...,가장 활성이 높은 형태: 인증된 유기농 올리브 오일에 제공되는 NatureWise ...,필수 비타민인 이유./비타민 D는 몸 전체의 세포에서 찾을 수 있습니다. 칼슘과 인...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
4,B06WRNG37C,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",Dr. Berg Nutritionals,37.99,35.50,4.7,14855,https://m.media-amazon.com/images/I/61TB8Sq3Hv...,"10,000IU의 비타민 D3 K2 보충제에는 비타민 D3 10,000IU(5,00...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
5,B09KCD4D7F,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",Centrum,19.99,15.99,4.7,40250,https://m.media-amazon.com/images/I/71kh-hE7hV...,"Centrum 실버 멀티비타민 남성용 50 플러스, 비타민 D3, B-비타민 및 아...",Centrum 남성용 실버 멀티비타민 50 플러스는 미량 영양소가 함유된 종합 비타...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
6,B07255MPRN,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,Sports Research,0.00,23.95,4.7,42932,https://m.media-amazon.com/images/I/512+wh1BkV...,"함께 더 좋음: ""햇빛 비타민""으로 알려진 비타민 D3는 칼슘 대사를 돕고 건강한 ...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
7,B09KCCVL9V,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",Centrum,19.29,15.66,4.8,19005,https://m.media-amazon.com/images/I/7150wfe2mj...,"Centrum Silver 여성용 멀티비타민 50 플러스 200개입 1병, 비타민 ...",Centrum Silver 여성용 멀티비타민 50 플러스는 전신 건강을 지원하는 미...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
8,B0741FTPXD,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",SmartyPants,32.99,24.76,4.5,14933,https://m.media-amazon.com/images/I/71Q4qdP6dg...,"포장은 다를 수 있습니다 - 신선하고 새로운 외관, 동일한 신뢰할 수 있는 품질 과...","SmartyPants Teen Guy Complete는 눈 건강을 위한 피쉬 오일,...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
9,B00DS5BGDY,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",Nature Made,23.89,9.98,4.7,33972,https://m.media-amazon.com/images/I/713dgE1NB6...,"Nature Made 비타민 B12 1000 mcg 소프트젤 150일 분량, 병당 ...",비타민 B12는 다른 비타민B와 마찬가지로 먹는 음식을 세포 에너지로 변환하는 데 ...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 B/B12"
11,B00I97GM12,"One A Day Teen for Him 멀티비타민 젤리, 면역 건강 지원, 신체 ...",ONE A DAY,0.00,9.98,4.6,8534,https://m.media-amazon.com/images/I/81Tv4gOtT6...,제품 참고: 열이나 햇빛에 노출되면 제품이 녹거나 손상될 수 있습니다. 따라서 고객...,One A Day For Him VitaCraves 틴 멀티비타민 젤리는 다양한 주...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"


In [22]:
# 중복 제거: 첫 번째 행만 유지
details_df_unique = detail_df.drop_duplicates(subset='title')

# '상품명'과 'title' 열을 기준으로 병합
merged_df = pd.merge(products_df, details_df_unique, left_on='상품명', right_on='title', how='inner')
merged_df = merged_df.reset_index(drop=True)
merged_df

,상품명,상품url,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,img_urls_paragraph,info_text,detail_paragraph,category_paragraph
0,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",https://www.amazon.com/-/ko/dp/B001F71XAI/ref=...,B001F71XAI,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",ONE A DAY,0.00,17.48,4.7,48490,https://m.media-amazon.com/images/I/616SPCJQAB...,One A Day 남성용 건강 포뮬라 멀티비타민은 심장의 건강과 같은 남성이 주로 ...,One A Day 남성용 건강 포뮬라 멀티비타민은 1개의 알로 다양한 주요 영양소를...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
1,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",https://www.amazon.com/-/ko/dp/B00YMSLT88/ref=...,B00YMSLT88,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",Nature Made,13.99,7.49,4.7,7463,https://m.media-amazon.com/images/I/71J+FRt+rp...,영양 지원 보충제: 철분이 함유된 Nature Made 멀티비타민 태블릿 130개입...,"Nature Made 철분 정제가 함유된 종합 비타민은 면역 건강, 근육 및 뼈 건...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
2,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",https://www.amazon.com/-/ko/dp/B00GB85JR4/ref=...,B00GB85JR4,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",NatureWise,0.00,14.99,4.7,174050,https://m.media-amazon.com/images/I/71JF8TOgXU...,가장 활성이 높은 형태: 인증된 유기농 올리브 오일에 제공되는 NatureWise ...,필수 비타민인 이유./비타민 D는 몸 전체의 세포에서 찾을 수 있습니다. 칼슘과 인...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
3,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",https://www.amazon.com/-/ko/dp/B06WRNG37C/ref=...,B06WRNG37C,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",Dr. Berg Nutritionals,37.99,35.50,4.7,14855,https://m.media-amazon.com/images/I/61TB8Sq3Hv...,"10,000IU의 비타민 D3 K2 보충제에는 비타민 D3 10,000IU(5,00...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
4,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCD4D7F/ref=...,B09KCD4D7F,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",Centrum,19.99,15.99,4.7,40250,https://m.media-amazon.com/images/I/71kh-hE7hV...,"Centrum 실버 멀티비타민 남성용 50 플러스, 비타민 D3, B-비타민 및 아...",Centrum 남성용 실버 멀티비타민 50 플러스는 미량 영양소가 함유된 종합 비타...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
5,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,https://www.amazon.com/-/ko/dp/B07255MPRN/ref=...,B07255MPRN,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,Sports Research,0.00,23.95,4.7,42932,https://m.media-amazon.com/images/I/512+wh1BkV...,"함께 더 좋음: ""햇빛 비타민""으로 알려진 비타민 D3는 칼슘 대사를 돕고 건강한 ...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
6,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCCVL9V/ref=...,B09KCCVL9V,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",Centrum,19.29,15.66,4.8,19005,https://m.media-amazon.com/images/I/7150wfe2mj...,"Centrum Silver 여성용 멀티비타민 50 플러스 200개입 1병, 비타민 ...",Centrum Silver 여성용 멀티비타민 50 플러스는 전신 건강을 지원하는 미...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
7,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",https://www.amazon.com/-/ko/dp/B0741FTPXD/ref=...,B0741FTPXD,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",SmartyPants,32.99,24.76,4.5,14933,https://m.media-amazon.com/images/I/71Q4qdP6dg...,"포장은 다를 수 있습니다 - 신선하고 새로운 외관, 동일한 신뢰할 수 있는 품질 과...","SmartyPants Teen Guy Complete는 눈 건강을 위한 피쉬 오일,...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
8,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",https://www.amazon.com/-/ko/dp/B00DS5BGDY/ref=...,B00DS5BGDY,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",Nature Made,23.89,9.98,4.7,33972,https://m.media-amazon.com/images/I/713dgE1NB6...,"Nature Made 비타민 B12 1000 mcg 소프트젤 150일 분량, 병당 ...",비타민 B12는 다른 비타민B와 마찬가지로 먹는 음식을 세포 에너지로 변환하는 데 ...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 B/B12"
9,"One A Day Teen for Him 멀티비타민 젤리, 면역 건강 지원, 신체 ...",https://www.amazon.com/-/ko/dp/B00I97GM12/ref=...,B00I97GM12,"One A Day Teen for Him 멀티비타민 젤리, 면역 건강 지원, 신체 ...",ONE A DAY,0.00,9.98,4.6,8534,https://m.media-amazon.com/images/I/81Tv4gOtT6...,제품 참고: 열이나 햇빛에 노출되면 제품이 녹거나 손상될 수 있습니다. 따라서 고객...,One A Day For Him VitaCraves 틴 멀티비타민 젤리는 다양한 주...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"


In [23]:
# discounted_price 컬럼에 1385를 곱해서 정수로 변환
processed_df = merged_df.copy()
processed_df['discounted_price'] = (processed_df['discounted_price'] * 1385).astype(int)

# listPrice 컬럼에 1385를 곱하고 정수로 변환
processed_df['original_price'] = (processed_df['original_price'] * 1385).astype(int)

# original_price 값이 0인 경우 discounted_price 컬럼의 값으로 대체 (판매가와 정가가 동일한 경우)
processed_df['original_price'] = processed_df.apply(
    lambda row: row['discounted_price'] if row['original_price'] == 0 else row['original_price'],
    axis=1
)
processed_df

,상품명,상품url,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,img_urls_paragraph,info_text,detail_paragraph,category_paragraph
0,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",https://www.amazon.com/-/ko/dp/B001F71XAI/ref=...,B001F71XAI,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",ONE A DAY,24209,24209,4.7,48490,https://m.media-amazon.com/images/I/616SPCJQAB...,One A Day 남성용 건강 포뮬라 멀티비타민은 심장의 건강과 같은 남성이 주로 ...,One A Day 남성용 건강 포뮬라 멀티비타민은 1개의 알로 다양한 주요 영양소를...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
1,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",https://www.amazon.com/-/ko/dp/B00YMSLT88/ref=...,B00YMSLT88,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",Nature Made,19376,10373,4.7,7463,https://m.media-amazon.com/images/I/71J+FRt+rp...,영양 지원 보충제: 철분이 함유된 Nature Made 멀티비타민 태블릿 130개입...,"Nature Made 철분 정제가 함유된 종합 비타민은 면역 건강, 근육 및 뼈 건...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
2,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",https://www.amazon.com/-/ko/dp/B00GB85JR4/ref=...,B00GB85JR4,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",NatureWise,20761,20761,4.7,174050,https://m.media-amazon.com/images/I/71JF8TOgXU...,가장 활성이 높은 형태: 인증된 유기농 올리브 오일에 제공되는 NatureWise ...,필수 비타민인 이유./비타민 D는 몸 전체의 세포에서 찾을 수 있습니다. 칼슘과 인...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
3,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",https://www.amazon.com/-/ko/dp/B06WRNG37C/ref=...,B06WRNG37C,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",Dr. Berg Nutritionals,52616,49167,4.7,14855,https://m.media-amazon.com/images/I/61TB8Sq3Hv...,"10,000IU의 비타민 D3 K2 보충제에는 비타민 D3 10,000IU(5,00...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
4,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCD4D7F/ref=...,B09KCD4D7F,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",Centrum,27686,22146,4.7,40250,https://m.media-amazon.com/images/I/71kh-hE7hV...,"Centrum 실버 멀티비타민 남성용 50 플러스, 비타민 D3, B-비타민 및 아...",Centrum 남성용 실버 멀티비타민 50 플러스는 미량 영양소가 함유된 종합 비타...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
5,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,https://www.amazon.com/-/ko/dp/B07255MPRN/ref=...,B07255MPRN,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,Sports Research,33170,33170,4.7,42932,https://m.media-amazon.com/images/I/512+wh1BkV...,"함께 더 좋음: ""햇빛 비타민""으로 알려진 비타민 D3는 칼슘 대사를 돕고 건강한 ...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D"
6,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCCVL9V/ref=...,B09KCCVL9V,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",Centrum,26716,21689,4.8,19005,https://m.media-amazon.com/images/I/7150wfe2mj...,"Centrum Silver 여성용 멀티비타민 50 플러스 200개입 1병, 비타민 ...",Centrum Silver 여성용 멀티비타민 50 플러스는 전신 건강을 지원하는 미...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
7,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",https://www.amazon.com/-/ko/dp/B0741FTPXD/ref=...,B0741FTPXD,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",SmartyPants,45691,34292,4.5,14933,https://m.media-amazon.com/images/I/71Q4qdP6dg...,"포장은 다를 수 있습니다 - 신선하고 새로운 외관, 동일한 신뢰할 수 있는 품질 과...","SmartyPants Teen Guy Complete는 눈 건강을 위한 피쉬 오일,...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"
8,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",https://www.amazon.com/-/ko/dp/B00DS5BGDY/ref=...,B00DS5BGDY,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",Nature Made,33087,13822,4.7,33972,https://m.media-amazon.com/images/I/713dgE1NB6...,"Nature Made 비타민 B12 1000 mcg 소프트젤 150일 분량, 병당 ...",비타민 B12는 다른 비타민B와 마찬가지로 먹는 음식을 세포 에너지로 변환하는 데 ...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 B/B12"
9,"One A Day Teen for Him 멀티비타민 젤리, 면역 건강 지원, 신체 ...",https://www.amazon.com/-/ko/dp/B00I97GM12/ref=...,B00I97GM12,"One A Day Teen for Him 멀티비타민 젤리, 면역 건강 지원, 신체 ...",ONE A DAY,13822,13822,4.6,8534,https://m.media-amazon.com/images/I/81Tv4gOtT6...,제품 참고: 열이나 햇빛에 노출되면 제품이 녹거나 손상될 수 있습니다. 따라서 고객...,One A Day For Him VitaCraves 틴 멀티비타민 젤리는 다양한 주...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민"


In [24]:
# '카테고리' 컬럼을 '/'로 나누기
category_split = processed_df['category_paragraph'].str.split('/', expand=True)

# 분리된 컬럼을 데이터 프레임에 추가
for i, col in enumerate(category_split.columns):
    processed_df[f'category_{i+1}'] = category_split[col]

# 결과 출력
processed_df

,상품명,상품url,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,img_urls_paragraph,info_text,detail_paragraph,category_paragraph,category_1,category_2,category_3,category_4,category_5
0,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",https://www.amazon.com/-/ko/dp/B001F71XAI/ref=...,B001F71XAI,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",ONE A DAY,24209,24209,4.7,48490,https://m.media-amazon.com/images/I/616SPCJQAB...,One A Day 남성용 건강 포뮬라 멀티비타민은 심장의 건강과 같은 남성이 주로 ...,One A Day 남성용 건강 포뮬라 멀티비타민은 1개의 알로 다양한 주요 영양소를...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,멀티비타민,None
1,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",https://www.amazon.com/-/ko/dp/B00YMSLT88/ref=...,B00YMSLT88,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",Nature Made,19376,10373,4.7,7463,https://m.media-amazon.com/images/I/71J+FRt+rp...,영양 지원 보충제: 철분이 함유된 Nature Made 멀티비타민 태블릿 130개입...,"Nature Made 철분 정제가 함유된 종합 비타민은 면역 건강, 근육 및 뼈 건...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,멀티비타민,None
2,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",https://www.amazon.com/-/ko/dp/B00GB85JR4/ref=...,B00GB85JR4,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",NatureWise,20761,20761,4.7,174050,https://m.media-amazon.com/images/I/71JF8TOgXU...,가장 활성이 높은 형태: 인증된 유기농 올리브 오일에 제공되는 NatureWise ...,필수 비타민인 이유./비타민 D는 몸 전체의 세포에서 찾을 수 있습니다. 칼슘과 인...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,비타민 D,None
3,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",https://www.amazon.com/-/ko/dp/B06WRNG37C/ref=...,B06WRNG37C,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",Dr. Berg Nutritionals,52616,49167,4.7,14855,https://m.media-amazon.com/images/I/61TB8Sq3Hv...,"10,000IU의 비타민 D3 K2 보충제에는 비타민 D3 10,000IU(5,00...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,비타민 D,None
4,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCD4D7F/ref=...,B09KCD4D7F,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",Centrum,27686,22146,4.7,40250,https://m.media-amazon.com/images/I/71kh-hE7hV...,"Centrum 실버 멀티비타민 남성용 50 플러스, 비타민 D3, B-비타민 및 아...",Centrum 남성용 실버 멀티비타민 50 플러스는 미량 영양소가 함유된 종합 비타...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,멀티비타민,None
5,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,https://www.amazon.com/-/ko/dp/B07255MPRN/ref=...,B07255MPRN,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,Sports Research,33170,33170,4.7,42932,https://m.media-amazon.com/images/I/512+wh1BkV...,"함께 더 좋음: ""햇빛 비타민""으로 알려진 비타민 D3는 칼슘 대사를 돕고 건강한 ...",,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 D",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,비타민 D,None
6,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCCVL9V/ref=...,B09KCCVL9V,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",Centrum,26716,21689,4.8,19005,https://m.media-amazon.com/images/I/7150wfe2mj...,"Centrum Silver 여성용 멀티비타민 50 플러스 200개입 1병, 비타민 ...",Centrum Silver 여성용 멀티비타민 50 플러스는 전신 건강을 지원하는 미...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,멀티비타민,None
7,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",https://www.amazon.com/-/ko/dp/B0741FTPXD/ref=...,B0741FTPXD,"SmartyPants Teen Guy Formula, 일일 종합비타민 젤리: 비타민...",SmartyPants,45691,34292,4.5,14933,https://m.media-amazon.com/images/I/71Q4qdP6dg...,"포장은 다를 수 있습니다 - 신선하고 새로운 외관, 동일한 신뢰할 수 있는 품질 과...","SmartyPants Teen Guy Complete는 눈 건강을 위한 피쉬 오일,...","건강 및 가정/비타민, 미네랄 및 영양제/비타민/멀티비타민",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,멀티비타민,None
8,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",https://www.amazon.com/-/ko/dp/B00DS5BGDY/ref=...,B00DS5BGDY,"Nature Made 비타민 B12 1000 mcg 소프트젤, 신진대사 건강을 위한...",Nature Made,33087,13822,4.7,33972,https://m.media-amazon.com/images/I/713dgE1NB6...,"Nature Made 비타민 B12 1000 mcg 소프트젤 150일 분량, 병당 ...",비타민 B12는 다른 비타민B와 마찬가지로 먹는 음식을 세포 에너지로 변환하는 데 ...,"건강 및 가정/비타민, 미네랄 및 영양제/비타민/비타민 B/B12",건강 및 가정,"비타민, 미네랄 및 영양제",비타민,비타민 B,B12
9,"One A Day Teen for Him 멀티비타민 젤리, 면역 건

In [25]:
imgUrl_split = processed_df['img_urls_paragraph'].str.split(' ', expand=True)

# 분리된 컬럼을 데이터 프레임에 추가
for i, col in enumerate(imgUrl_split.columns):
    processed_df[f'imgUrl_{i+1}'] = imgUrl_split[col]

# 결과 출력
processed_df

,상품명,상품url,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,img_urls_paragraph,...,category_3,category_4,category_5,imgUrl_1,imgUrl_2,imgUrl_3,imgUrl_4,imgUrl_5,imgUrl_6,imgUrl_7
0,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",https://www.amazon.com/-/ko/dp/B001F71XAI/ref=...,B001F71XAI,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",ONE A DAY,24209,24209,4.7,48490,https://m.media-amazon.com/images/I/616SPCJQAB...,...,비타민,멀티비타민,None,https://m.media-amazon.com/images/I/616SPCJQAB...,https://m.media-amazon.com/images/I/81sD0SLKIo...,https://m.media-amazon.com/images/I/71Hcrp07kI...,https://m.media-amazon.com/images/I/81thndBP16...,https://m.media-amazon.com/images/I/71rKxvcKT8...,https://m.media-amazon.com/images/I/71PHnGUTBD...,None
1,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",https://www.amazon.com/-/ko/dp/B00YMSLT88/ref=...,B00YMSLT88,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",Nature Made,19376,10373,4.7,7463,https://m.media-amazon.com/images/I/71J+FRt+rp...,...,비타민,멀티비타민,None,https://m.media-amazon.com/images/I/71J+FRt+rp...,https://m.media-amazon.com/images/I/81aV8bLqIg...,https://m.media-amazon.com/images/I/81EJcGPjNg...,https://m.media-amazon.com/images/I/61-2se3-oq...,https://m.media-amazon.com/images/I/71QwHohvnk...,https://m.media-amazon.com/images/I/71N0SdybhD...,None
2,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",https://www.amazon.com/-/ko/dp/B00GB85JR4/ref=...,B00GB85JR4,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",NatureWise,20761,20761,4.7,174050,https://m.media-amazon.com/images/I/71JF8TOgXU...,...,비타민,비타민 D,None,https://m.media-amazon.com/images/I/71JF8TOgXU...,https://m.media-amazon.com/images/I/81BmCqLBHG...,https://m.media-amazon.com/images/I/81yFrsts9H...,https://m.media-amazon.com/images/I/71KuUdgtWx...,https://m.media-amazon.com/images/I/81A1mXYykF...,https://m.media-amazon.com/images/I/81O256D5Qy...,None
3,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",https://www.amazon.com/-/ko/dp/B06WRNG37C/ref=...,B06WRNG37C,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",Dr. Berg Nutritionals,52616,49167,4.7,14855,https://m.media-amazon.com/images/I/61TB8Sq3Hv...,...,비타민,비타민 D,None,https://m.media-amazon.com/images/I/61TB8Sq3Hv...,https://m.media-amazon.com/images/I/71udj3AlK2...,https://m.media-amazon.com/images/I/71ORL8DBsc...,https://m.media-amazon.com/images/I/71eia9DWDg...,https://m.media-amazon.com/images/I/71OL0yTncz...,https://m.media-amazon.com/images/I/81w0OTZCtw...,None
4,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCD4D7F/ref=...,B09KCD4D7F,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",Centrum,27686,22146,4.7,40250,https://m.media-amazon.com/images/I/71kh-hE7hV...,...,비타민,멀티비타민,None,https://m.media-amazon.com/images/I/71kh-hE7hV...,https://m.media-amazon.com/images/I/81TXf92+2w...,https://m.media-amazon.com/images/I/71TaTES8HF...,https://m.media-amazon.com/images/I/71wsnjhCXd...,https://m.media-amazon.com/images/I/81euW9dEfA...,https://m.media-amazon.com/images/I/81znOhNWUU...,None
5,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,https://www.amazon.com/-/ko/dp/B07255MPRN/ref=...,B07255MPRN,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,Sports Research,33170,33170,4.7,42932,https://m.media-amazon.com/images/I/512+wh1BkV...,...,비타민,비타민 D,None,https://m.media-amazon.com/images/I/512+wh1BkV...,https://m.media-amazon.com/images/I/714igAlOg8...,https://m.media-amazon.com/images/I/71v+9Z5abZ...,https://m.media-amazon.com/images/I/71cWiohPPN...,https://m.media-amazon.com/images/I/81YF3wGZFl...,https://m.media-amazon.com/images/I/71e32O9n2q...,None
6,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",https://www.amazon.com/-/ko/dp/B09KCCVL9V/ref=...,B09KCCVL9V,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",Centrum,26716,21689,4.8,19005,https://m.media-amazon.com/images/I/7150wfe2mj...,...,비타민,멀티비타민,None,https://m.media-amazon.com/images/I/7150wfe2mj...,https://m.media-amazon.com/i

In [26]:
processed_df = processed_df.drop(columns=['상품명', 'img_urls_paragraph', 'category_paragraph'])
processed_df

,상품url,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,info_text,detail_paragraph,...,category_3,category_4,category_5,imgUrl_1,imgUrl_2,imgUrl_3,imgUrl_4,imgUrl_5,imgUrl_6,imgUrl_7
0,https://www.amazon.com/-/ko/dp/B001F71XAI/ref=...,B001F71XAI,"One A Day 남성용 멀티비타민, 비타민 A, 비타민 C, 비타민 D, 비타민 ...",ONE A DAY,24209,24209,4.7,48490,One A Day 남성용 건강 포뮬라 멀티비타민은 심장의 건강과 같은 남성이 주로 ...,One A Day 남성용 건강 포뮬라 멀티비타민은 1개의 알로 다양한 주요 영양소를...,...,비타민,멀티비타민,None,https://m.media-amazon.com/images/I/616SPCJQAB...,https://m.media-amazon.com/images/I/81sD0SLKIo...,https://m.media-amazon.com/images/I/71Hcrp07kI...,https://m.media-amazon.com/images/I/81thndBP16...,https://m.media-amazon.com/images/I/71rKxvcKT8...,https://m.media-amazon.com/images/I/71PHnGUTBD...,None
1,https://www.amazon.com/-/ko/dp/B00YMSLT88/ref=...,B00YMSLT88,"Nature Made 철분이 함유된 멀티비타민 태블릿, 여성 및 남성용 종합 비타민...",Nature Made,19376,10373,4.7,7463,영양 지원 보충제: 철분이 함유된 Nature Made 멀티비타민 태블릿 130개입...,"Nature Made 철분 정제가 함유된 종합 비타민은 면역 건강, 근육 및 뼈 건...",...,비타민,멀티비타민,None,https://m.media-amazon.com/images/I/71J+FRt+rp...,https://m.media-amazon.com/images/I/81aV8bLqIg...,https://m.media-amazon.com/images/I/81EJcGPjNg...,https://m.media-amazon.com/images/I/61-2se3-oq...,https://m.media-amazon.com/images/I/71QwHohvnk...,https://m.media-amazon.com/images/I/71N0SdybhD...,None
2,https://www.amazon.com/-/ko/dp/B00GB85JR4/ref=...,B00GB85JR4,"NatureWise 비타민 D3 5000IU(125mcg), 1년분, 건강한 근육 ...",NatureWise,20761,20761,4.7,174050,가장 활성이 높은 형태: 인증된 유기농 올리브 오일에 제공되는 NatureWise ...,필수 비타민인 이유./비타민 D는 몸 전체의 세포에서 찾을 수 있습니다. 칼슘과 인...,...,비타민,비타민 D,None,https://m.media-amazon.com/images/I/71JF8TOgXU...,https://m.media-amazon.com/images/I/81BmCqLBHG...,https://m.media-amazon.com/images/I/81yFrsts9H...,https://m.media-amazon.com/images/I/71KuUdgtWx...,https://m.media-amazon.com/images/I/81A1mXYykF...,https://m.media-amazon.com/images/I/81O256D5Qy...,None
3,https://www.amazon.com/-/ko/dp/B06WRNG37C/ref=...,B06WRNG37C,"Dr. Berg 비타민 D3 K2 MCT 파우더 포함 - 비타민 D3 10,000I...",Dr. Berg Nutritionals,52616,49167,4.7,14855,"10,000IU의 비타민 D3 K2 보충제에는 비타민 D3 10,000IU(5,00...",,...,비타민,비타민 D,None,https://m.media-amazon.com/images/I/61TB8Sq3Hv...,https://m.media-amazon.com/images/I/71udj3AlK2...,https://m.media-amazon.com/images/I/71ORL8DBsc...,https://m.media-amazon.com/images/I/71eia9DWDg...,https://m.media-amazon.com/images/I/71OL0yTncz...,https://m.media-amazon.com/images/I/81w0OTZCtw...,None
4,https://www.amazon.com/-/ko/dp/B09KCD4D7F/ref=...,B09KCD4D7F,"Centrum Silver 남성용 50+ 멀티비타민 비타민 비타민 D3, 비타민 B...",Centrum,27686,22146,4.7,40250,"Centrum 실버 멀티비타민 남성용 50 플러스, 비타민 D3, B-비타민 및 아...",Centrum 남성용 실버 멀티비타민 50 플러스는 미량 영양소가 함유된 종합 비타...,...,비타민,멀티비타민,None,https://m.media-amazon.com/images/I/71kh-hE7hV...,https://m.media-amazon.com/images/I/81TXf92+2w...,https://m.media-amazon.com/images/I/71TaTES8HF...,https://m.media-amazon.com/images/I/71wsnjhCXd...,https://m.media-amazon.com/images/I/81euW9dEfA...,https://m.media-amazon.com/images/I/81znOhNWUU...,None
5,https://www.amazon.com/-/ko/dp/B07255MPRN/ref=...,B07255MPRN,Sports Research 비건 비타민 D3 + K2 보충제 유기농 코코넛 오일 ...,Sports Research,33170,33170,4.7,42932,"함께 더 좋음: ""햇빛 비타민""으로 알려진 비타민 D3는 칼슘 대사를 돕고 건강한 ...",,...,비타민,비타민 D,None,https://m.media-amazon.com/images/I/512+wh1BkV...,https://m.media-amazon.com/images/I/714igAlOg8...,https://m.media-amazon.com/images/I/71v+9Z5abZ...,https://m.media-amazon.com/images/I/71cWiohPPN...,https://m.media-amazon.com/images/I/81YF3wGZFl...,https://m.media-amazon.com/images/I/71e32O9n2q...,None
6,https://www.amazon.com/-/ko/dp/B09KCCVL9V/ref=...,B09KCCVL9V,"Centrum Silver 여성용 멀티비타민 50 플러스, 비타민 D3, 비타민 B...",Centrum,26716,21689,4.8,19005,"Centrum Silver 여성용 멀티비타민 50 플러스 200개입 1병, 비타민 ...",Centrum Silver 여성용 멀티비타민 50 플러스는 전신 건강을 지원하는 미...,...,비타민,멀티비타민,None,https://m.media-amazon.com/images/I/7150wfe2mj...,https://m.media-amazon.com/images/I/71CJPReQog...,https://m.media-amazon.com/images/I/81lvTTQFl6...,https://m.media-amazon

In [27]:
processed_df.to_csv('비타민_data.csv', index=False, encoding='utf-8-sig')

# 3. 이미지 파일 크롤링

상품 ID는 규칙을 만들어 새로 지정할 예정 

아마존의 기존 상품 ID는 이미지 파일명으로 사용
- asin 값인 상품ID_1, 상품ID_2, 상품ID_3 ... 순서로 이미지 파일명을 지정하여 이미지 수집

In [28]:
def download_images(df):
    for index, row in df.iterrows():
        asin = row['asin']
        
        for i in range(1, 8):
            image_url_col = f'imgUrl_{i}'
            
            if image_url_col in row and pd.notna(row[image_url_col]):
                image_url = row[image_url_col]
                
                try:
                    # 이미지 요청
                    response = requests.get(image_url)
                    response.raise_for_status()  # 요청이 성공했는지 확인
                    
                    # 이미지 열기
                    image = Image.open(BytesIO(response.content))
                    
                    # 이미지 저장 경로 설정 및 저장
                    image_filename = os.path.join(SAVE_DIR, f'{asin}_{i}.jpg')
                    image.save(image_filename)
                    print(f"{image_filename} 저장 완료.")
                
                except Exception as e:
                    print(f"{image_url_col}에서 이미지를 다운로드하는 동안 오류가 발생했습니다: {e}")

In [29]:
# 함수 실행
download_images(processed_df)

C:\Users\yejis\AisayHi data\images\B001F71XAI_1.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B001F71XAI_2.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B001F71XAI_3.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B001F71XAI_4.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B001F71XAI_5.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B001F71XAI_6.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00YMSLT88_1.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00YMSLT88_2.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00YMSLT88_3.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00YMSLT88_4.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00YMSLT88_5.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00YMSLT88_6.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00GB85JR4_1.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00GB85JR4_2.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00GB85JR4_3.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00GB85JR4_4.jpg 저장 완료.
C:\Users\yejis\AisayHi data\images\B00GB85JR4_5.jpg 저장 완